In [4]:
import os
import matplotlib.pyplot as plt
import torchvision
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils.notebook import display_scrollable_dataframe,plot_sailency
from data_loaders import CUB_extnded_dataset
from models import get_inception_transform
from IPython.display import display

from sailency import get_saliency_maps,saliency_score_part

%reload_ext autoreload
%autoreload 2

# Data analysis 
On the use of Majority voting on the concetps. First the data is loaded. 

In [ ]:
# Settings for the experiment your running

# Settings for loading the dataset, make sure its the same as the one used to train the model. 
model_folder_MV = r"models\Sequential_Basemodel3"
model_folder_NoMV = r"models\Sequential_Basemodel2"


use_majority_voting_MV = True
use_majority_voting_NoMV = False

# You can find the settings in the .hydra folders config.yaml 
data_config_MV = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_MV,
                'min_class_count':10,
                'return_visibility':True}


# You can find the settings in the .hydra folders config.yaml 
data_config_NoMV = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_NoMV,
                'min_class_count':10,
                'return_visibility':True}

#Load models make sure the model is trained with the same settings as the data loader

X_to_C_path = os.path.join(model_folder_MV,"best_XtoC_model.pth")
C_to_Y_path = os.path.join(model_folder_MV,"best_CtoY_model.pth")

ModelXtoC_MV = torch.load(X_to_C_path,map_location=torch.device('cpu'))
ModelCtoY_MV = torch.load(C_to_Y_path,map_location=torch.device('cpu'))

X_to_C_path = os.path.join(model_folder_NoMV,"best_XtoC_model.pth")
C_to_Y_path = os.path.join(model_folder_NoMV,"best_CtoY_model.pth")

ModelXtoC_NoMV = torch.load(X_to_C_path,map_location=torch.device('cpu'))
ModelCtoY_NoMV = torch.load(C_to_Y_path,map_location=torch.device('cpu'))

In [6]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

112 312


In [7]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts and visibility
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts and visibility
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [8]:
data_set = 'train' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer)
data_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer)

#Get a dataset that return original images instead normalized ones
data_human_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform)
concept_names_MV_train = data_MV_train.consept_labels_names
class_names_MV_train = data_MV_train.class_labels_names
n_classes_MV_train = data_MV_train.n_classes
n_concepts_MV_train = data_MV_train.n_concepts

data_human_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform)
concept_names_NoMV_train = data_NoMV_train.consept_labels_names
class_names_NoMV_train = data_NoMV_train.class_labels_names
n_classes_NoMV_train = data_NoMV_train.n_classes
n_concepts_NoMV_train = data_NoMV_train.n_concepts


print(n_concepts_MV_train,n_concepts_NoMV_train)

112 312


In [9]:
print(data_MV_train[0][0].shape) # image
print(data_MV_train[0][1].shape) # concepts and visibility
print(data_MV_train[0][2].shape) # classes
print(len(data_MV_train[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_train[0][0].shape) # image
print(data_NoMV_train[0][1].shape) # concepts and visibility
print(data_NoMV_train[0][2].shape) # classes
print(len(data_NoMV_train[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [11]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, reduce = False)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, reduce = False)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, reduce = False)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, reduce = False)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

312 312


In [12]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([312])
torch.Size([200])
312
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [13]:
data_set = 'train' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, reduce = False)
data_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, reduce = False)

#Get a dataset that return original images instead normalized ones
data_human_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, reduce = False)
concept_names_MV_train = data_MV_train.consept_labels_names
class_names_MV_train = data_MV_train.class_labels_names
n_classes_MV_train = data_MV_train.n_classes
n_concepts_MV_train = data_MV_train.n_concepts

data_human_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, reduce = False)
concept_names_NoMV_train = data_NoMV_train.consept_labels_names
class_names_NoMV_train = data_NoMV_train.class_labels_names
n_classes_NoMV_train = data_NoMV_train.n_classes
n_concepts_NoMV_train = data_NoMV_train.n_concepts


print(n_concepts_MV_train,n_concepts_NoMV_train)

312 312


In [14]:
print(data_MV_train[0][0].shape) # image
print(data_MV_train[0][1].shape) # concepts
print(data_MV_train[0][2].shape) # classes
print(len(data_MV_train[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_train[0][0].shape) # image
print(data_NoMV_train[0][1].shape) # concepts
print(data_NoMV_train[0][2].shape) # classes
print(len(data_NoMV_train[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([312])
torch.Size([200])
312
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [15]:
# compare all the concepts in data_MV_train to data_noMV_train
same = 0
for item_MV, item_NoMV in zip(tqdm(data_MV_val), data_NoMV_val):

    same_i = torch.sum(item_MV[1] == item_NoMV[1][0])
    same += same_i


print(same)
print(len(data_MV_val)*312)
print(same/(len(data_MV_val)*312))


100%|██████████| 1198/1198 [00:40<00:00, 29.26it/s]


tensor(373776)
373776
tensor(1.)


With our data loader using the non reduced data, the MV and NoMV validation sets are exactly the same!

In [16]:
# compare all the concepts in data_MV_train to data_noMV_train
same = 0
for item_MV, item_NoMV in zip(tqdm(data_MV_train), data_NoMV_train):

    same_i = torch.sum(item_MV[1] == item_NoMV[1][0])
    same += same_i

print(same)
print(len(data_MV_train)*312)
print(same/(len(data_MV_train)*312))


100%|██████████| 4796/4796 [02:29<00:00, 32.12it/s]

tensor(1347208)
1496352
tensor(0.9003)


With our dataloader using the non reduced data, 90% of the MV training set is the same as the NoMV training set. Thus we lost 10% of information.

This could be noise, also it could just be variations in the bird species such as male and female ducks looking very different. Would be interesting to look into, but I don't know if we have time.

In [31]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, reduce = False, andreas=True)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, reduce = False, andreas=True)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, reduce = False, andreas=True)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, reduce = False, andreas=True)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

312 312


In [18]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([312])
torch.Size([200])
312
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [ ]:
data_set = 'train' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, reduce = False, andreas=True)
data_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, reduce = False, andreas=True)

#Get a dataset that return original images instead normalized ones
data_human_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, reduce = False, andreas=True)
concept_names_MV_train = data_MV_train.consept_labels_names
class_names_MV_train = data_MV_train.class_labels_names
n_classes_MV_train = data_MV_train.n_classes
n_concepts_MV_train = data_MV_train.n_concepts

data_human_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, reduce = False, andreas=True)
concept_names_NoMV_train = data_NoMV_train.consept_labels_names
class_names_NoMV_train = data_NoMV_train.class_labels_names
n_classes_NoMV_train = data_NoMV_train.n_classes
n_concepts_NoMV_train = data_NoMV_train.n_concepts


print(n_concepts_MV_train,n_concepts_NoMV_train)

312 312


In [20]:
print(data_MV_train[0][0].shape) # image
print(data_MV_train[0][1].shape) # concepts
print(data_MV_train[0][2].shape) # classes
print(len(data_MV_train[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_train[0][0].shape) # image
print(data_NoMV_train[0][1].shape) # concepts
print(data_NoMV_train[0][2].shape) # classes
print(len(data_NoMV_train[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([312])
torch.Size([200])
312
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [21]:
# compare all the concepts in data_MV_train to data_noMV_train
same = 0
for item_MV, item_NoMV in zip(tqdm(data_MV_val), data_NoMV_val):

    same_i = torch.sum(item_MV[1] == item_NoMV[1][0])
    same += same_i


print(same)
print(len(data_MV_val)*312)
print(same/(len(data_MV_val)*312))


100%|██████████| 1198/1198 [00:26<00:00, 44.43it/s]

tensor(336328)
373776
tensor(0.8998)


With Andreas' dataloader using the non reduced data, only 90% of the MV validation set is the same as the NoMV validation set. Thus we lost 10% of information. This is the same as for the training set as we expected and this does not make sense to do, since you are not suppose to know the true class of the validation set and thus you cannot perform MV on it.

In [22]:
# compare all the concepts in data_MV_train to data_noMV_train
same = 0
for item_MV, item_NoMV in zip(tqdm(data_MV_train), data_NoMV_train):

    same_i = torch.sum(item_MV[1] == item_NoMV[1][0])
    same += same_i

print(same)
print(len(data_MV_train)*312)
print(same/(len(data_MV_train)*312))


100%|██████████| 4796/4796 [01:28<00:00, 54.15it/s]

tensor(1347208)
1496352
tensor(0.9003)


Again with Andreas' dataloader using the non reduced data, only 90% of the MV training set is the same as the NoMV training set. Thus we lost 10% of information. This is exactly the same af for our dataloader, which it is suppose to as it should not have changed the training set only the validation set.

In [23]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, andreas=True)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, andreas=True)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, andreas=True)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, andreas=True)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

112 312


In [24]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [25]:
data_set = 'train' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, andreas=True)
data_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, andreas=True)

#Get a dataset that return original images instead normalized ones
data_human_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, andreas=True)
concept_names_MV_train = data_MV_train.consept_labels_names
class_names_MV_train = data_MV_train.class_labels_names
n_classes_MV_train = data_MV_train.n_classes
n_concepts_MV_train = data_MV_train.n_concepts

data_human_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, andreas=True)
concept_names_NoMV_train = data_NoMV_train.consept_labels_names
class_names_NoMV_train = data_NoMV_train.class_labels_names
n_classes_NoMV_train = data_NoMV_train.n_classes
n_concepts_NoMV_train = data_NoMV_train.n_concepts


print(n_concepts_MV_train,n_concepts_NoMV_train)

112 312


In [26]:
print(data_MV_train[0][0].shape) # image
print(data_MV_train[0][1].shape) # concepts
print(data_MV_train[0][2].shape) # classes
print(len(data_MV_train[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_train[0][0].shape) # image
print(data_NoMV_train[0][1].shape) # concepts
print(data_NoMV_train[0][2].shape) # classes
print(len(data_NoMV_train[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [32]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val_andreas = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, andreas=True)

concept_names_MV_val_andreas = data_MV_val_andreas.consept_labels_names
class_names_MV_val_andreas = data_MV_val_andreas.class_labels_names
n_classes_MV_val_andreas = data_MV_val_andreas.n_classes
n_concepts_MV_val_andreas = data_MV_val_andreas.n_concepts

print(n_concepts_MV_val_andreas)

112


In [33]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer)

concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts


print(n_concepts_MV_val)

112


In [35]:
# compare all the concepts in data_MV_train to data_noMV_train
same = 0
for item_MV, item_MV_andreas in zip(tqdm(data_MV_val), data_MV_val_andreas):

    same_i = torch.sum(item_MV[1] == item_MV_andreas[1])
    same += same_i

print(same)
print(len(data_MV_val)*312)
print(same/(len(data_MV_val)*312))


100%|██████████| 1198/1198 [00:41<00:00, 29.10it/s]

tensor(109327)
373776
tensor(0.2925)


Andreas' validation set and our validations set are very different! Now this is because we're now only looking at the reduced dataset, where we have looked at the non reduced dataset until now. The non reduced dataset contains a lot of zeros that will be the same for the no matter if MV is applied or not, thus when we are only looking at the reduced dataset the difference becomes more evident.